In [ ]:
import os, numpy as np, tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# load data
codes, lines = list(), 1000
for root, dirs, files in os.walk('fake tensorflow'):
	for file in files:
		if file.find('.') != -1 and file.split('.')[1] == 'py':
			file = open(os.path.join(root, file), 'r')
			for line in file:
				if len(line) > 4:
					codes.append(line)
					lines -= 1
				if lines == 0:
					break
			file.close()
		if lines == 0:
			break
	if lines == 0:
		break

In [ ]:
codes = ''.join(codes)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(codes)
n_steps, stride = 100, 1
X, Y = list(), list()
for i in range(0, len(codes) - n_steps, stride):
	X.append(tokenizer.texts_to_matrix(codes[i: i + n_steps], 'tfidf'))
	Y.append(tokenizer.texts_to_matrix(codes[i + stride: i + n_steps + stride], 'tfidf'))
X, Y = np.array(X), np.array(Y)
del codes

In [ ]:
def build_model():
	input_layer = tf.keras.Input(shape=(n_steps, len(tokenizer.index_word) + 1))
	norm = tf.keras.layers.LayerNormalization()(input_layer)
	lstm = tf.keras.layers.LSTM(256, return_sequences=True, kernel_regularizer='l2')(norm)
	dense = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer='l2')(lstm)
	norm = tf.keras.layers.LayerNormalization()(dense)
	attention = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=4, kernel_regularizer='l2')(norm, norm)
	output_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tokenizer.index_word) + 1, activation='softmax'))(attention)
	model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
	model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
	return model

In [ ]:
model = build_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=2)
model.fit(X, Y, validation_split=.2, epochs=10, callbacks=[early_stopping])
model.save('model')

Epoch 1/10
1162/1162 [==============================] - 478s 409ms/step - loss: 4.8117 - accuracy: 0.3535 - val_loss: 6.1971 - val_accuracy: 0.3527
Epoch 2/10
1162/1162 [==============================] - 473s 407ms/step - loss: 1.7936 - accuracy: 0.6054 - val_loss: 7.6396 - val_accuracy: 0.3659
Epoch 3/10
1162/1162 [==============================] - 473s 407ms/step - loss: 1.1533 - accuracy: 0.6531 - val_loss: 7.9081 - val_accuracy: 0.3696
Epoch 00003: early stopping


INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


overfitting happened because the training set is not large enough. there is no more memory to generate more data. so, the code generation is poor.

In [9]:
query = '''
def enable():
  # Enables v2 behaviors.
  _pywrap_tf2.enable(True)

def disable():
  # Disables v2 behaviors.
  _pywrap_tf2.enable(False)

@tf_export("__internal__.tf2.enabled", v1=[])
def enabled():
  # Returns True iff TensorFlow 2.0 behavior should be enabled.
  return _pywrap_tf2.is_enabled()'''
query = tokenizer.texts_to_matrix(query[:100], 'tfidf')
predict = model.predict(query.reshape([-1, query.shape[0], query.shape[1]])).squeeze()
predict = [np.argmax(one_hot) for one_hot in predict]
print(tokenizer.sequences_to_texts([predict]))

["s e f a f n c b l e d d f v v c c c c n a b l e s o v 2 b b e h a v i o r s a o ' p p p y w r a p t t f 2 0 e n a b l e ' e r u e n d d d e f i f i s a b l e s v v v n n d d d i s a b l e s e v 2 b b"]
